<a name='2'></a>
# Part 2: LSTMs for POS tagging - using PyTorch (40 Points)
In this part, We will be building a bidirectional LSTM network to train and inference POS tagging on UDPOS dataset.<br>

PyTorch makes it easy by abstracting most of the details that go in building,training and inferencing a neural network. We recommend going through every PyTorch function that this notebook uses to gain more understanding.   

If you need a refresher or have never worked with Neural Networks before, here are a few resources:
- https://web.stanford.edu/~jurafsky/slp3/7.pdf
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/

We will be using PyTorch for defining, training and inferencing a neural network for our POS Tagging problem. If you have not used any deep learning framework/library, we recommend you spend some time understanding how to use these libraries. 

PyTorch Resources:
- https://pytorch.org/tutorials/
- https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html 

You will need the following imports. Install these libraries using the following commands. 
- Installing pytorch - https://pytorch.org/get-started/locally/ (choose your setup from here)
- conda install -c conda-forge spacy
- conda install -c pytorch torchtext

Training a neural network model will take time. 
- Make use of your **Nvidia** GPU if you have one. 
- If not, you can use Google Colab / Kaggle notebooks. You get a free GPU for a limited time to tweak your hyperparameters.
- Without a GPU, You might have to wait longer to experiment.

In [1]:
!pip install torch==1.8.0
!pip install torchtext==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 994.6 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchmetrics 0.11.1 requires torch>=1.8.1, but you have torch 1.8.0 which is incompatible.
pytorch-lightning 1.9.3 requires torch>=1.10.0, but you have torch 1.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.0
    Uninstalling torchtext-0.14.0:
      Successfully uninstalled torchtext-0.14.0


In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim

# a package that provides processing utilities and popular datasets for natural language

from torchtext.legacy import data
from torchtext.legacy.datasets import UDPOS

import spacy
from tqdm import tqdm 
import random
import numpy as np

In [3]:
# Using a seed to maintain consistent and reproducible results
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# This cell downloads and prepares data, a TorchText Dataset Object

TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
fields = (("text", TEXT), ("udtags", UD_TAGS))
train_data, valid_data, test_data = UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:00<00:00, 2.27MB/s]


extracting


### Visualizing the torchtext dataset

In [5]:
print("Length of the dataset", len(train_data))
for i in range(0,5):
    print("TEXT ", i+1 ,*(train_data[i].__dict__['text']))
    print("TAGS ", i+1 ,*(train_data[i].__dict__['udtags']))

Length of the dataset 12543
TEXT  1 al - zaman : american forces killed shaikh abdullah al - ani , the preacher at the mosque in the town of qaim , near the syrian border .
TAGS  1 PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT
TEXT  2 [ this killing of a respected cleric will be causing us trouble for years to come . ]
TAGS  2 PUNCT DET NOUN ADP DET ADJ NOUN AUX AUX VERB PRON NOUN ADP NOUN PART VERB PUNCT PUNCT
TEXT  3 dpa : iraqi authorities announced that they had busted up 3 terrorist cells operating in baghdad .
TAGS  3 PROPN PUNCT ADJ NOUN VERB SCONJ PRON AUX VERB ADP NUM ADJ NOUN VERB ADP PROPN PUNCT
TEXT  4 two of them were being run by 2 officials of the ministry of the interior !
TAGS  4 NUM ADP PRON AUX AUX VERB ADP NUM NOUN ADP DET PROPN ADP DET PROPN PUNCT
TEXT  5 the moi in iraq is equivalent to the us fbi , so this would be like having j. edgar hoover unwittingly employ a

### GloVe Vector initialization 
Vectorizing the input words is an impotant step in the NLP pipeline that can determine the end performance of neural networks. GloVe vectors capture both global statistics and local statistics of a corpus. We use GloVe to convert words to embeddings in the vector space based on their semantics. 

To learn more about GloVe please read the following resource:
- https://nlp.stanford.edu/pubs/glove.pdf

In [6]:
# the words should have atleast a min frequency of 2 to build its vocab
MIN_FREQ = 2

# Torch text builds the vocabulary based on word representations from glove. 
TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)


UD_TAGS.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:15<00:00, 25958.64it/s]


In [7]:
# number of tags in the dataset
len(UD_TAGS.vocab)

18

##### Expected output
18

<a name='2.1'></a>
# Part 2.1: Building the neural network

We will make use of the GloVe embeddings and build a bi-directional LSTM. You will be able to tune the hyper parameters of the network and see what works. 

It involves duplicating the first recurrent layer in the network so that there are now two layers side-by-side, then providing the input sequence as-is as input to the first layer and providing a reversed copy of the input sequence to the second.

The idea is to split the state neurons of a regular RNN in a part that is responsible for the positive time direction (forward states) and a part for the negative time direction (backward states)

More on it here: https://maxwell.ict.griffith.edu.au/spl/publications/papers/ieeesp97_schuster.pdf

All the internal computations/details will be taken care by PyTorch. You will be able to implement many variations of this neural networks with minor changes in code. Expect your neural network definition to be under 10 lines.

Your PyTorch model (inherits torch.nn.Module) definition contains defining two functions:
    -Init : Which specifies what layers to initialize.
    -Forward: Which defines the order of computations in these layers. <br>
**Note** - We will not grade based on accuracy, We grade if your model converges. You can follow your order of code, if you think the comments are not helping.  

### Part 2.1.2 Building LSTM network - 20 Points

In [8]:
class LSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        # Define an embedding layer that converts the words to embeddings based on GloVe.
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        
        # Define a bi-directional LSTM layer with the hyperparameters. 
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)       
        
        # Define a dropout layer that helps in regularization
        self.dropout = nn.Dropout(dropout)
        
        # Define a Linear layer which can associate lstm output to the final output 
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        
    def forward(self, text):
        # pass text through embedding layer
        embedded = self.dropout(self.embedding(text))
        # pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded)

        # pass the LSTM output to dropout and fully connected linear layer
        embedded_dropout = self.dropout(outputs)
        predictions = self.fc(embedded_dropout)
        
        # we use our outputs to make a prediction of what the tag should be
        # predictions = [sent len, batch size, output dim]
      
        return predictions

In [9]:
# Tweak the Nones
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [10]:
# initializing model weights for better convergence
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, std=0.1)
model.apply(init_weights)

# initializing model embeddings with glove word vectors
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# making the padding embeddings as all zero, as we don't want to learn paddings.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [11]:
# If your PC doesn't have enough CPU Ram or Video memory, try decreasing the batch_size
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# BucketIterator allows for data to be split into buckets of equal size,
# any remaining space is filled with pad token
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

### Optimizer and Loss Function
Optimizers are algorithms or methods used to change the attributes of the neural network such as weights and learning rate to reduce the losses. Optimizers are used to solve optimization problems by minimizing the function.
- PyTorch provides many Optimizer Algorithms, feel free to try them and the one that works best for you. 
- Link - https://pytorch.org/docs/stable/optim.html
- We will be using CrossEntropyLoss as predicting a word tag is a classification problem.

In [13]:
# optimizer to train the model
optimizer = optim.Adam(model.parameters(), lr=0.001)
# ignoring the padding in our loss calculation
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [14]:
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [15]:
# use gpu if available, These lines move your model to gpu from cpu if available
model = model.to(device)
criterion = criterion.to(device)

# If this line prints cuda, your machine is equipped with a Nvidia GPU and PyTorch is utilizing the GPU
print(device)

cuda


In [16]:
# method to check for accurcy of the model ignoring the pad index
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != TAG_PAD_IDX).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

### Part 2.1.2 - Training and Testing LSTM - 20 Points
- Decide your epochs to train based on loss and accuracy
- Fill single line PyTorch commands. 

In [17]:
EPOCHS = 20

for epoch in tqdm(range(EPOCHS)):
    model.train()
    train_epoch_loss = 0
    train_epoch_acc = 0
    print(f'Epoch: {epoch+1:02}\n')
    for batch in train_iterator:
        
        # returns a batch of text to train on (sent len, batch size)
        text = batch.text
        tags = batch.udtags
        
        # Add a command that makes the optimizer with zero gradients for each iteration 
        optimizer.zero_grad()

        
        # Add a command that feeds the batch to the model
        predictions = model(text)

        
        # predictions = (sent len, batch size, output dim)
          # tags = (sent len, batch size)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        # Add a command that calculates loss
        loss = criterion(predictions, tags)

        
        # Make use of the categorical accuracy and calculate accuracy
        acc = categorical_accuracy(predictions, tags)

        
        # Add a command that calculates gradients
        loss.backward()

        
        # Add a command that updates the weights by taking steps 
        optimizer.step()

        # Calculate loss and accuracy for the epoch
        train_epoch_loss += loss.item()
        train_epoch_acc += acc.item()

    train_epoch_loss /= len(train_iterator)
    train_epoch_acc /= len(train_iterator)
                  
     # Calculate loss
    print(f'\t [Train Loss] : {train_epoch_loss:.3f} | [Train Acc] : {train_epoch_acc*100:.2f}%\n')
    
  
    val_epoch_loss = 0
    val_epoch_acc = 0 

    # Add a command that moves the model to validation mode
    model.eval()
    
    with torch.no_grad():
    
        for batch in valid_iterator:

            text = batch.text
            tags = batch.udtags
        
            # Add the same command that feeds the batch to the model
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            # Add the same command that calculates loss
            val_loss = criterion(predictions, tags)

            # Make use of the categorical accuracy function and calculate accuracy
            val_acc = categorical_accuracy(predictions, tags)

            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()

  # Calculate validation loss
    
    val_epoch_loss /= len(valid_iterator)
    val_epoch_acc /= len(valid_iterator)

    print(f'\t [Val Loss] : {val_loss:.3f} | [Val Acc] : {val_acc*100:.2f}%\n')      

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 01



  5%|▌         | 1/20 [00:05<01:36,  5.06s/it]

	 [Train Loss] : 1.448 | [Train Acc] : 53.91%

	 [Val Loss] : 0.602 | [Val Acc] : 80.95%

Epoch: 02



 10%|█         | 2/20 [00:09<01:27,  4.84s/it]

...

	 [Train Loss] : 0.173 | [Train Acc] : 94.34%

	 [Val Loss] : 0.247 | [Val Acc] : 92.26%

Epoch: 19



 95%|█████████▌| 19/20 [01:29<00:04,  4.69s/it]

	 [Train Loss] : 0.168 | [Train Acc] : 94.49%

	 [Val Loss] : 0.250 | [Val Acc] : 92.17%

Epoch: 20



100%|██████████| 20/20 [01:33<00:00,  4.70s/it]

	 [Train Loss] : 0.162 | [Train Acc] : 94.65%

	 [Val Loss] : 0.251 | [Val Acc] : 92.17%



In [18]:
# testing the accuracy on test set
test_acc=0
model.eval()

# Computes without the gradients. Use this while testing your model.
# As we do not intend to learn from the data
with torch.no_grad():
    for batch in test_iterator:
      
      text = batch.text
      tags = batch.udtags

    # Add the same command that feeds the batch to the model
      predictions = model(text)

      predictions = predictions.view(-1, predictions.shape[-1])
      tags = tags.view(-1)

    # Make use of the categorical accuracy function and calculate accuracy
      test_acc += categorical_accuracy(predictions, tags)
    
# Calculate Accuracy

final_acc = test_acc / len(test_iterator)
print(f'Test Acc: {final_acc*100:.2f}%\n')

Test Acc: 89.91%



***Note***: You are using a different dataset compared to part 1 and 2. This part of the assignment is designed/aimed to help develop basic understanding of Neural Networks. Although we expect an accuracy of above 85%, we do not grade based on the accuracy output. 

In [19]:
# Try different inputs to these function.
def test_lstm(test_sentence):
    x= test_sentence.unsqueeze(-1).to(device)
    pred = model(x)
    pred = pred.argmax(-1)
    pred_tags = [UD_TAGS.vocab.itos[t.item()] for t in pred]
    true_tags = [UD_TAGS.vocab.itos[t.item()] for t in test_labels]
    tokenized_sentence = [TEXT.vocab.itos[t.item()] for t in test_sentence]
    return tokenized_sentence, true_tags,pred_tags    

In [20]:
test_sentence = text[0]
test_labels = tags[0:len(test_sentence)]
print(test_lstm(test_sentence)[0])
print("True tags", test_lstm(test_sentence)[1])
print("Predicted Tags",test_lstm(test_sentence)[2])

['(', 'stay', 'important', 'the', '<unk>', 'i', 'did', '<unk>', 'the', 'here', 'we', 'as', 'i', 'on', '<unk>', 'it', 'it', 'just', '"', 'the', 'the', '"', 'if', 'the', 'any', 'in', 'in', 'over', 'it']
True tags ['PUNCT', 'VERB', 'ADJ', 'DET', 'ADV', 'PRON', 'AUX', 'PROPN', 'DET', 'ADV', 'PRON', 'SCONJ', 'PRON', 'ADP', 'PROPN', 'PRON', 'PRON', 'ADV', 'PUNCT', 'DET', 'DET', 'PUNCT', 'SCONJ', 'DET', 'DET', 'ADP', 'ADP', 'ADV', 'PRON']
Predicted Tags ['PUNCT', 'VERB', 'ADJ', 'DET', 'NOUN', 'PRON', 'AUX', 'VERB', 'DET', 'ADV', 'PRON', 'SCONJ', 'PRON', 'SCONJ', 'VERB', 'PRON', 'PRON', 'ADV', 'PUNCT', 'DET', 'DET', 'PUNCT', 'SCONJ', 'DET', 'DET', 'ADP', 'ADV', 'ADP', 'PRON']


In [21]:
# Save your model if it performs well. This saves all the trained weights,
# so that you don't have to train again in your codewalk
torch.save(model.state_dict(), "./LSTMPOSTAG.pth")

# loading?l
# model.load_state_dict(torch.load("./LSTMPOSTAG.pth"))

### Theory Questions: 10 Points
*** Q1. Give some real word examples where POS tagging is used   ***<br>
*** Q2. What are the hidden variables in HMM in this assignment? Why are they called hidden? *** <br>
*** Q3. How Viterbi Algorithm provides more efficient estimation compared to brute force calculation of all tag combinations? *** <br>

## Q1
1. Text-to-speech (TTS) systems: POS tagging is used to determine the correct pronunciation and intonation of words in a text-to-speech system. By identifying the part of speech of each word, the TTS system can determine the correct stress and emphasis to use when speaking the word.

2. Information retrieval: POS tagging can be used to improve search results in information retrieval systems. By identifying the part of speech of each word in a query, the system can better understand the user's intent and retrieve more relevant documents.

3. Sentiment analysis: POS tagging is used to identify the parts of speech that are most associated with positive or negative sentiment in a text. By analyzing the frequency and distribution of these parts of speech, sentiment analysis models can classify the overall sentiment of a piece of text.

4. Machine translation: POS tagging is used to improve the accuracy of machine translation systems. By identifying the part of speech of each word in the source language, the system can better understand the structure and meaning of the sentence and generate more accurate translations.

5. Named entity recognition: POS tagging is used to identify named entities such as people, places, and organizations in a text. By identifying the part of speech of each word in a sentence, named entity recognition models can more accurately identify and classify these entities.

## Q2
HMMs (Hidden Markov Models) are a type of probabilistic graphical model that can predict a series of unknown (hidden) variables given a set of observed variables. The terms "states" and "observations" are used interchangeably to refer to the hidden and observable states, respectively.

Transition data – the likelihood of shifting to a new state based on the current state.

Emission data – the likelihood of moving from a hidden state to an observed state.

Initial state information — the initial probability of transitioning to a hidden state. This might also be referred to as the likelihood previous to the event. The data or emission matrix is conditioned on the hidden state, which is C(ti), which is the number of times tags appeared in the training data (stored in the tag counts dictionary).

## Q3
The Viterbi algorithm provides a more efficient estimation compared to the brute-force calculation of all tag combinations by breaking down the problem into smaller subproblems and avoiding redundant calculations. Instead of calculating all possible tag sequences, the algorithm recursively computes the maximum probability of a sequence of hidden states that ends in a given state at each position in the sequence. It keeps track of the maximum probability of a partial sequence of hidden states up to each position and the most likely tag that led to that maximum probability. By using the results of previous subproblems to efficiently compute the current subproblem, the algorithm avoids redundant calculations and significantly reduces the computation time required to estimate the most likely sequence of hidden states. This makes it possible to find the most likely sequence of hidden states even for long sequences and large state spaces in linear time.